In [21]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import os
import numpy as np
from PIL import Image

# Dataset directory
data_dir = r'E:\Abroad period research\Medical images analysis paper implementation codes\Second part of the paper\testing on Generated Eye Dataset for Glaucoma Detection\10 features results\Acrima'

# Parameters
IMG_SIZE = (150, 150)
BATCH_SIZE = 32
EPOCHS = 30
TEST_SIZE = 0.15  # 15% for testing

# Load image paths and labels
def load_image_paths_and_labels(base_dir):
    class_names = sorted([d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))])
    all_filepaths = []
    all_labels = []
    
    for idx, class_name in enumerate(class_names):
        class_folder = os.path.join(base_dir, class_name)
        file_names = [f for f in os.listdir(class_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        
        for file_name in file_names:
            all_filepaths.append(os.path.join(class_folder, file_name))
            all_labels.append(idx)  # numeric label
            
    return np.array(all_filepaths), np.array(all_labels), class_names

filepaths, labels, class_names = load_image_paths_and_labels(data_dir)
print(f"Total images: {len(filepaths)} | Classes: {class_names}")

# Split data
train_paths, test_paths, train_labels, test_labels = train_test_split(
    filepaths, labels, test_size=TEST_SIZE, stratify=labels, random_state=42
)

print(f"Train samples: {len(train_paths)} | Test samples: {len(test_paths)}")

# Preprocess function
def load_and_preprocess_image(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)  # handle jpeg/png
    image = tf.image.resize(image, IMG_SIZE)
    image = image / 255.0  # normalize to [0,1]
    return image, label

# TF datasets
train_ds = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
train_ds = train_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((test_paths, test_labels))
test_ds = test_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

# Model setup
base_model = tf.keras.applications.ResNet50(
    input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False  # freeze

# Number of classes
num_classes = len(class_names)
print(f"Number of classes: {num_classes}")

# Final activation and loss
if num_classes == 2:
    final_activation = 'sigmoid'
    final_units = 1
    loss_function = 'binary_crossentropy'
else:
    final_activation = 'softmax'
    final_units = num_classes
    loss_function = 'sparse_categorical_crossentropy'  # because labels are integers

# Build the model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(final_units, activation=final_activation)
])

# Compile
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=loss_function,
    metrics=['accuracy']
)

# Train
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=test_ds
)

# Save the model
model.save('glaucoma_detection_resnet50.h5')

# Evaluate & Predict
# Get test data arrays for predictions
test_images = []
test_labels_list = []

for image, label in test_ds.unbatch():
    test_images.append(image.numpy())
    test_labels_list.append(label.numpy())

test_images = np.array(test_images)
test_labels_list = np.array(test_labels_list)

# Predictions
predictions = model.predict(test_images)

if num_classes == 2:
    predicted_classes = (predictions > 0.5).astype("int32").flatten()
else:
    predicted_classes = np.argmax(predictions, axis=1)

# Classification report
report = classification_report(test_labels_list, predicted_classes, target_names=class_names)
print("Classification Report:\n", report)

# Final validation accuracy
val_acc = history.history['val_accuracy'][-1]
print(f"Final validation accuracy: {val_acc:.4f}")


Total images: 6000 | Classes: ['Glaucoma', 'Normal']
Train samples: 5100 | Test samples: 900
Number of classes: 2
Epoch 1/30
160/160 ━━━━━━━━━━━━━━━━━━━━ 287s 2s/step - accuracy: 0.4924 - loss: 0.7378 - val_accuracy: 0.6644 - val_loss: 0.6816
Epoch 2/30
160/160 ━━━━━━━━━━━━━━━━━━━━ 216s 1s/step - accuracy: 0.5321 - loss: 0.6897 - val_accuracy: 0.6211 - val_loss: 0.6798
Epoch 3/30
160/160 ━━━━━━━━━━━━━━━━━━━━ 245s 2s/step - accuracy: 0.5659 - loss: 0.6847 - val_accuracy: 0.5656 - val_loss: 0.6753
Epoch 4/30
160/160 ━━━━━━━━━━━━━━━━━━━━ 246s 2s/step - accuracy: 0.5593 - loss: 0.6813 - val_accuracy: 0.6011 - val_loss: 0.6662
Epoch 5/30
160/160 ━━━━━━━━━━━━━━━━━━━━ 244s 2s/step - accuracy: 0.5718 - loss: 0.6758 - val_accuracy: 0.6800 - val_loss: 0.6518
Epoch 6/30
160/160 ━━━━━━━━━━━━━━━━━━━━ 249s 2s/step - accuracy: 0.5835 - loss: 0.6713 - val_accuracy: 0.6767 - val_loss: 0.6463
Epoch 7/30
160/160 ━━━━━━━━━━━━━━━━━━━━ 244s 2s/step - accuracy: 0.5991 - loss: 0.6624 - val_accuracy: 0.6856 - 

29/29 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step
Classification Report:
               precision    recall  f1-score   support

    Glaucoma       0.71      0.64      0.67       450
      Normal       0.67      0.75      0.71       450

    accuracy                           0.69       900
   macro avg       0.69      0.69      0.69       900
weighted avg       0.69      0.69      0.69       900

Final validation accuracy: 0.6911


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import os
import numpy as np
from PIL import Image

# Dataset directory
data_dir = r'E:\Abroad period research\Medical images analysis paper implementation codes\Second part of the paper\testing code on brain tumor dataset\dataset'

# Parameters
IMG_SIZE = (150, 150)
BATCH_SIZE = 32
EPOCHS = 30
TEST_SIZE = 0.15  # 15% for testing

# Load image paths and labels
def load_image_paths_and_labels(base_dir):
    class_names = sorted([d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))])
    all_filepaths = []
    all_labels = []
    
    for idx, class_name in enumerate(class_names):
        class_folder = os.path.join(base_dir, class_name)
        file_names = [f for f in os.listdir(class_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        
        for file_name in file_names:
            all_filepaths.append(os.path.join(class_folder, file_name))
            all_labels.append(idx)  # numeric label
            
    return np.array(all_filepaths), np.array(all_labels), class_names

filepaths, labels, class_names = load_image_paths_and_labels(data_dir)
print(f"Total images: {len(filepaths)} | Classes: {class_names}")

# Split data
train_paths, test_paths, train_labels, test_labels = train_test_split(
    filepaths, labels, test_size=TEST_SIZE, stratify=labels, random_state=42
)

print(f"Train samples: {len(train_paths)} | Test samples: {len(test_paths)}")

# Preprocess function
def load_and_preprocess_image(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)  # handle jpeg/png
    image = tf.image.resize(image, IMG_SIZE)
    image = image / 255.0  # normalize to [0,1]
    return image, label

# TF datasets
train_ds = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
train_ds = train_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((test_paths, test_labels))
test_ds = test_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

# Model setup
base_model = tf.keras.applications.ResNet50(
    input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False  # freeze

# Number of classes
num_classes = len(class_names)
print(f"Number of classes: {num_classes}")

# Final activation and loss
if num_classes == 2:
    final_activation = 'sigmoid'
    final_units = 1
    loss_function = 'binary_crossentropy'
else:
    final_activation = 'softmax'
    final_units = num_classes
    loss_function = 'sparse_categorical_crossentropy'  # because labels are integers

# Build the model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(final_units, activation=final_activation)
])

# Compile
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=loss_function,
    metrics=['accuracy']
)

# Train
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=test_ds
)

# Save the model
model.save('glaucoma_detection_resnet50.h5')

# Evaluate & Predict
# Get test data arrays for predictions
test_images = []
test_labels_list = []

for image, label in test_ds.unbatch():
    test_images.append(image.numpy())
    test_labels_list.append(label.numpy())

test_images = np.array(test_images)
test_labels_list = np.array(test_labels_list)

# Predictions
predictions = model.predict(test_images)

if num_classes == 2:
    predicted_classes = (predictions > 0.5).astype("int32").flatten()
else:
    predicted_classes = np.argmax(predictions, axis=1)

# Classification report
report = classification_report(test_labels_list, predicted_classes, target_names=class_names, digits=4)
print("Classification Report:\n", report)

# Final validation accuracy
val_acc = history.history['val_accuracy'][-1]
print(f"Final validation accuracy: {val_acc:.4f}")


Total images: 2400 | Classes: ['Benign', 'Malignant']
Train samples: 2040 | Test samples: 360
Number of classes: 2
Epoch 1/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 97s 1s/step - accuracy: 0.5108 - loss: 0.7331 - val_accuracy: 0.7333 - val_loss: 0.6554
Epoch 2/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 84s 1s/step - accuracy: 0.6022 - loss: 0.6679 - val_accuracy: 0.6056 - val_loss: 0.6307
Epoch 3/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 89s 1s/step - accuracy: 0.6580 - loss: 0.6382 - val_accuracy: 0.7889 - val_loss: 0.5978
Epoch 4/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 89s 1s/step - accuracy: 0.6962 - loss: 0.6111 - val_accuracy: 0.7278 - val_loss: 0.5809
Epoch 5/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 92s 1s/step - accuracy: 0.7231 - loss: 0.5729 - val_accuracy: 0.7417 - val_loss: 0.5420
Epoch 6/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 98s 2s/step - accuracy: 0.7534 - loss: 0.5444 - val_accuracy: 0.7694 - val_loss: 0.5093
Epoch 7/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 93s 1s/step - accuracy: 0.7472 - loss: 0.5283 - val_accuracy: 0.8306 - val_loss: 0.4803
Epo

12/12 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step 
Classification Report:
               precision    recall  f1-score   support

      Benign       0.85      0.94      0.89       180
   Malignant       0.94      0.83      0.88       180

    accuracy                           0.89       360
   macro avg       0.89      0.89      0.89       360
weighted avg       0.89      0.89      0.89       360

Final validation accuracy: 0.8889


Lung and Colon Cancer Histopathological Images ResNet50

In [23]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import os
import numpy as np
from PIL import Image

# Dataset directory
data_dir = r'E:\Abroad period research\Medical images analysis paper implementation codes\Second part of the paper\lung_colon_image_set\dataset'

# Parameters
IMG_SIZE = (150, 150)
BATCH_SIZE = 32
EPOCHS = 30
TEST_SIZE = 0.15  # 15% for testing

# Load image paths and labels
def load_image_paths_and_labels(base_dir):
    class_names = sorted([d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))])
    all_filepaths = []
    all_labels = []
    
    for idx, class_name in enumerate(class_names):
        class_folder = os.path.join(base_dir, class_name)
        file_names = [f for f in os.listdir(class_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        
        for file_name in file_names:
            all_filepaths.append(os.path.join(class_folder, file_name))
            all_labels.append(idx)  # numeric label
            
    return np.array(all_filepaths), np.array(all_labels), class_names

filepaths, labels, class_names = load_image_paths_and_labels(data_dir)
print(f"Total images: {len(filepaths)} | Classes: {class_names}")

# Split data
train_paths, test_paths, train_labels, test_labels = train_test_split(
    filepaths, labels, test_size=TEST_SIZE, stratify=labels, random_state=42
)

print(f"Train samples: {len(train_paths)} | Test samples: {len(test_paths)}")

# Preprocess function
def load_and_preprocess_image(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)  # handle jpeg/png
    image = tf.image.resize(image, IMG_SIZE)
    image = image / 255.0  # normalize to [0,1]
    return image, label

# TF datasets
train_ds = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
train_ds = train_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((test_paths, test_labels))
test_ds = test_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

# Model setup
base_model = tf.keras.applications.ResNet50(
    input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False  # freeze

# Number of classes
num_classes = len(class_names)
print(f"Number of classes: {num_classes}")

# Final activation and loss
if num_classes == 2:
    final_activation = 'sigmoid'
    final_units = 1
    loss_function = 'binary_crossentropy'
else:
    final_activation = 'softmax'
    final_units = num_classes
    loss_function = 'sparse_categorical_crossentropy'  # because labels are integers

# Build the model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(final_units, activation=final_activation)
])

# Compile
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=loss_function,
    metrics=['accuracy']
)

# Train
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=test_ds
)

# Save the model
model.save('glaucoma_detection_resnet50.h5')

# Evaluate & Predict
# Get test data arrays for predictions
test_images = []
test_labels_list = []

for image, label in test_ds.unbatch():
    test_images.append(image.numpy())
    test_labels_list.append(label.numpy())

test_images = np.array(test_images)
test_labels_list = np.array(test_labels_list)

# Predictions
predictions = model.predict(test_images)

if num_classes == 2:
    predicted_classes = (predictions > 0.5).astype("int32").flatten()
else:
    predicted_classes = np.argmax(predictions, axis=1)

# Classification report
report = classification_report(test_labels_list, predicted_classes, target_names=class_names, digits=4)
print("Classification Report:\n", report)

# Final validation accuracy
val_acc = history.history['val_accuracy'][-1]
print(f"Final validation accuracy: {val_acc:.4f}")


Total images: 25000 | Classes: ['colon_aca', 'colon_n', 'lung_aca', 'lung_n', 'lung_scc']
Train samples: 21250 | Test samples: 3750
Number of classes: 5
Epoch 1/30
665/665 ━━━━━━━━━━━━━━━━━━━━ 1606s 2s/step - accuracy: 0.2923 - loss: 1.5604 - val_accuracy: 0.5571 - val_loss: 1.2238
Epoch 2/30
665/665 ━━━━━━━━━━━━━━━━━━━━ 1694s 3s/step - accuracy: 0.4546 - loss: 1.2542 - val_accuracy: 0.5608 - val_loss: 1.0485
Epoch 3/30
665/665 ━━━━━━━━━━━━━━━━━━━━ 1000s 1s/step - accuracy: 0.4847 - loss: 1.1467 - val_accuracy: 0.5893 - val_loss: 0.9634
Epoch 4/30
665/665 ━━━━━━━━━━━━━━━━━━━━ 967s 1s/step - accuracy: 0.5167 - loss: 1.0895 - val_accuracy: 0.5885 - val_loss: 0.9344
Epoch 5/30
665/665 ━━━━━━━━━━━━━━━━━━━━ 1010s 2s/step - accuracy: 0.5258 - loss: 1.0539 - val_accuracy: 0.6141 - val_loss: 0.9026
Epoch 6/30
665/665 ━━━━━━━━━━━━━━━━━━━━ 935s 1s/step - accuracy: 0.5492 - loss: 1.0200 - val_accuracy: 0.6064 - val_loss: 0.9279
Epoch 7/30
665/665 ━━━━━━━━━━━━━━━━━━━━ 937s 1s/step - accuracy: 0.54

118/118 ━━━━━━━━━━━━━━━━━━━━ 156s 1s/step
Classification Report:
               precision    recall  f1-score   support

   colon_aca     0.7849    0.5693    0.6600       750
     colon_n     0.6772    0.8253    0.7440       750
    lung_aca     0.6643    0.5013    0.5714       750
      lung_n     0.7467    0.8413    0.7912       750
    lung_scc     0.7503    0.8813    0.8105       750

    accuracy                         0.7237      3750
   macro avg     0.7247    0.7237    0.7154      3750
weighted avg     0.7247    0.7237    0.7154      3750

Final validation accuracy: 0.7237


Ultrasound Breast Images for Breast Cancer ResNet50

In [26]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import os
import numpy as np
from PIL import Image

# Dataset directory
data_dir = r'E:\Abroad period research\Medical images analysis paper implementation codes\Second part of the paper\Ultrasound Breast Images for Breast Cancer\ultrasound breast classification'

# Parameters
IMG_SIZE = (150, 150)
BATCH_SIZE = 32
EPOCHS = 30
TEST_SIZE = 0.15  # 15% for testing

# Load image paths and labels
def load_image_paths_and_labels(base_dir):
    class_names = sorted([d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))])
    all_filepaths = []
    all_labels = []
    
    for idx, class_name in enumerate(class_names):
        class_folder = os.path.join(base_dir, class_name)
        file_names = [f for f in os.listdir(class_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        
        for file_name in file_names:
            all_filepaths.append(os.path.join(class_folder, file_name))
            all_labels.append(idx)  # numeric label
            
    return np.array(all_filepaths), np.array(all_labels), class_names

filepaths, labels, class_names = load_image_paths_and_labels(data_dir)
print(f"Total images: {len(filepaths)} | Classes: {class_names}")

# Split data
train_paths, test_paths, train_labels, test_labels = train_test_split(
    filepaths, labels, test_size=TEST_SIZE, stratify=labels, random_state=42
)

print(f"Train samples: {len(train_paths)} | Test samples: {len(test_paths)}")

# Preprocess function
def load_and_preprocess_image(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)  # handle jpeg/png
    image = tf.image.resize(image, IMG_SIZE)
    image = image / 255.0  # normalize to [0,1]
    return image, label

# TF datasets
train_ds = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
train_ds = train_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((test_paths, test_labels))
test_ds = test_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

# Model setup
base_model = tf.keras.applications.ResNet50(
    input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False  # freeze

# Number of classes
num_classes = len(class_names)
print(f"Number of classes: {num_classes}")

# Final activation and loss
if num_classes == 2:
    final_activation = 'sigmoid'
    final_units = 1
    loss_function = 'binary_crossentropy'
else:
    final_activation = 'softmax'
    final_units = num_classes
    loss_function = 'sparse_categorical_crossentropy'  # because labels are integers

# Build the model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(final_units, activation=final_activation)
])

# Compile
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=loss_function,
    metrics=['accuracy']
)

# Train
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=test_ds
)

# Save the model
model.save('glaucoma_detection_resnet50.h5')

# Evaluate & Predict
# Get test data arrays for predictions
test_images = []
test_labels_list = []

for image, label in test_ds.unbatch():
    test_images.append(image.numpy())
    test_labels_list.append(label.numpy())

test_images = np.array(test_images)
test_labels_list = np.array(test_labels_list)

# Predictions
predictions = model.predict(test_images)

if num_classes == 2:
    predicted_classes = (predictions > 0.5).astype("int32").flatten()
else:
    predicted_classes = np.argmax(predictions, axis=1)

# Classification report
report = classification_report(test_labels_list, predicted_classes, target_names=class_names, digits=4)
print("Classification Report:\n", report)

# Final validation accuracy
val_acc = history.history['val_accuracy'][-1]
print(f"Final validation accuracy: {val_acc:.4f}")


Total images: 9016 | Classes: ['benign', 'malignant']
Train samples: 7663 | Test samples: 1353
Number of classes: 2
Epoch 1/30
240/240 ━━━━━━━━━━━━━━━━━━━━ 374s 1s/step - accuracy: 0.5730 - loss: 0.6871 - val_accuracy: 0.7066 - val_loss: 0.6148
Epoch 2/30
240/240 ━━━━━━━━━━━━━━━━━━━━ 328s 1s/step - accuracy: 0.6695 - loss: 0.6144 - val_accuracy: 0.7317 - val_loss: 0.5608
Epoch 3/30
240/240 ━━━━━━━━━━━━━━━━━━━━ 334s 1s/step - accuracy: 0.7141 - loss: 0.5702 - val_accuracy: 0.7243 - val_loss: 0.5457
Epoch 4/30
240/240 ━━━━━━━━━━━━━━━━━━━━ 335s 1s/step - accuracy: 0.7399 - loss: 0.5477 - val_accuracy: 0.7421 - val_loss: 0.5195
Epoch 5/30
240/240 ━━━━━━━━━━━━━━━━━━━━ 335s 1s/step - accuracy: 0.7457 - loss: 0.5333 - val_accuracy: 0.7310 - val_loss: 0.5301
Epoch 6/30
240/240 ━━━━━━━━━━━━━━━━━━━━ 338s 1s/step - accuracy: 0.7486 - loss: 0.5251 - val_accuracy: 0.7664 - val_loss: 0.5018
Epoch 7/30
240/240 ━━━━━━━━━━━━━━━━━━━━ 341s 1s/step - accuracy: 0.7487 - loss: 0.5240 - val_accuracy: 0.7857 

43/43 ━━━━━━━━━━━━━━━━━━━━ 49s 1s/step
Classification Report:
               precision    recall  f1-score   support

      benign     0.8025    0.8586    0.8296       686
   malignant     0.8433    0.7826    0.8118       667

    accuracy                         0.8211      1353
   macro avg     0.8229    0.8206    0.8207      1353
weighted avg     0.8226    0.8211    0.8208      1353

Final validation accuracy: 0.8211


COVID-19 Radiography Database ResNet50

In [25]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import os
import numpy as np
from PIL import Image

# Dataset directory
data_dir = r'E:\Abroad period research\Medical images analysis paper implementation codes\Second part of the paper\Results on augmented just chest dataset\justchest_Unet_Segmented_Dataset'

# Parameters
IMG_SIZE = (150, 150)
BATCH_SIZE = 32
EPOCHS = 30
TEST_SIZE = 0.15  # 15% for testing

# Load image paths and labels
def load_image_paths_and_labels(base_dir):
    class_names = sorted([d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))])
    all_filepaths = []
    all_labels = []
    
    for idx, class_name in enumerate(class_names):
        class_folder = os.path.join(base_dir, class_name)
        file_names = [f for f in os.listdir(class_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        
        for file_name in file_names:
            all_filepaths.append(os.path.join(class_folder, file_name))
            all_labels.append(idx)  # numeric label
            
    return np.array(all_filepaths), np.array(all_labels), class_names

filepaths, labels, class_names = load_image_paths_and_labels(data_dir)
print(f"Total images: {len(filepaths)} | Classes: {class_names}")

# Split data
train_paths, test_paths, train_labels, test_labels = train_test_split(
    filepaths, labels, test_size=TEST_SIZE, stratify=labels, random_state=42
)

print(f"Train samples: {len(train_paths)} | Test samples: {len(test_paths)}")

# Preprocess function
def load_and_preprocess_image(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)  # handle jpeg/png
    image = tf.image.resize(image, IMG_SIZE)
    image = image / 255.0  # normalize to [0,1]
    return image, label

# TF datasets
train_ds = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
train_ds = train_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((test_paths, test_labels))
test_ds = test_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

# Model setup
base_model = tf.keras.applications.ResNet50(
    input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False  # freeze

# Number of classes
num_classes = len(class_names)
print(f"Number of classes: {num_classes}")

# Final activation and loss
if num_classes == 2:
    final_activation = 'sigmoid'
    final_units = 1
    loss_function = 'binary_crossentropy'
else:
    final_activation = 'softmax'
    final_units = num_classes
    loss_function = 'sparse_categorical_crossentropy'  # because labels are integers

# Build the model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(final_units, activation=final_activation)
])

# Compile
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=loss_function,
    metrics=['accuracy']
)

# Train
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=test_ds
)

# Save the model
model.save('glaucoma_detection_resnet50.h5')

# Evaluate & Predict
# Get test data arrays for predictions
test_images = []
test_labels_list = []

for image, label in test_ds.unbatch():
    test_images.append(image.numpy())
    test_labels_list.append(label.numpy())

test_images = np.array(test_images)
test_labels_list = np.array(test_labels_list)

# Predictions
predictions = model.predict(test_images)

if num_classes == 2:
    predicted_classes = (predictions > 0.5).astype("int32").flatten()
else:
    predicted_classes = np.argmax(predictions, axis=1)

# Classification report
report = classification_report(test_labels_list, predicted_classes, target_names=class_names, digits=4)
print("Classification Report:\n", report)

# Final validation accuracy
val_acc = history.history['val_accuracy'][-1]
print(f"Final validation accuracy: {val_acc:.4f}")


Total images: 40768 | Classes: ['COVID_Augmented', 'Lung_Opacity_Augmented', 'Normal', 'Viral Pneumonia_Augmented']
Train samples: 34652 | Test samples: 6116
Number of classes: 4
Epoch 1/30
1083/1083 ━━━━━━━━━━━━━━━━━━━━ 1620s 1s/step - accuracy: 0.3773 - loss: 1.2874 - val_accuracy: 0.5448 - val_loss: 1.0528
Epoch 2/30
1083/1083 ━━━━━━━━━━━━━━━━━━━━ 1487s 1s/step - accuracy: 0.4953 - loss: 1.0837 - val_accuracy: 0.5878 - val_loss: 0.9812
Epoch 3/30
1083/1083 ━━━━━━━━━━━━━━━━━━━━ 1505s 1s/step - accuracy: 0.5455 - loss: 1.0005 - val_accuracy: 0.6537 - val_loss: 0.8789
Epoch 4/30
1083/1083 ━━━━━━━━━━━━━━━━━━━━ 1498s 1s/step - accuracy: 0.5811 - loss: 0.9432 - val_accuracy: 0.6624 - val_loss: 0.8264
Epoch 5/30
1083/1083 ━━━━━━━━━━━━━━━━━━━━ 1603s 1s/step - accuracy: 0.6016 - loss: 0.9063 - val_accuracy: 0.6687 - val_loss: 0.7979
Epoch 6/30
1083/1083 ━━━━━━━━━━━━━━━━━━━━ 1511s 1s/step - accuracy: 0.6099 - loss: 0.8861 - val_accuracy: 0.6915 - val_loss: 0.7715
Epoch 7/30
1083/1083 ━━━━━━━━

192/192 ━━━━━━━━━━━━━━━━━━━━ 268s 1s/step
Classification Report:
                            precision    recall  f1-score   support

          COVID_Augmented     0.6632    0.3774    0.4810      1529
   Lung_Opacity_Augmented     0.6154    0.5736    0.5938      1529
                   Normal     0.8036    0.9529    0.8719      1529
Viral Pneumonia_Augmented     0.7052    0.9261    0.8007      1529

                 accuracy                         0.7075      6116
                macro avg     0.6969    0.7075    0.6869      6116
             weighted avg     0.6969    0.7075    0.6869      6116

Final validation accuracy: 0.7075


Resnet50